# Estimating Gentrification using Street View Images and Embeddings

This script (initially produced by ChatGPT) does the following (_this was my query_):
 - Read a spatial boundary file (that I will hard code)
 - Obtain the road network (from OSM?) for that area
 - Generate sample points on the road network roughly X meters apart
 - At each sample point, download the most recent street images for that location (either a single 360 degree view of a few smaller images). Use whichever API service is the most appropriate for obtaining the images. Importantly please record the date that the image was taken.
 - For each image, calculate an embedding using an appropriate foundation model (one that has been pre-trained to distinguish street environments specifically). Please use Hugging Face libraries.
 - If necessary, calculate the mean embedding for each point (is this the best way to calculate a single embedding for a point represented by multiple images?)
 - Now, for each sampled point there will be a dataframe with information about the point and its embedding. Read another polygon spatial data file, that I will provide, which contains area-level estimates of gentrification.
 - Use point-in-polygon to get the gentrification for each point.
 - Use cross-validation to train a couple of ML models (probaly random forest, linear regression and a neural network) to estimate gentrification from the embedding vectors
 - Choose the best model and parameter configuration and test this model on some held-out data.

## Configuration and library loading

In [1]:
import os
os.environ["HF_HOME"] = "/nfs/a319/gy17m2a/scratch/hf_cache"
import pickle
import torch
from PIL import Image
from tqdm import tqdm
import clip

data_dir = os.path.join("../../../data/embeddings/")

### Load list of sample points

In [4]:
# Cache file for the entire points data with embeddings (images are stored separately)
DEBUG = False
points_data_cache = data_dir + "sample_points_cache/points_data_cache_with_embeddings.pkl"

# -----------------------------------------------------------
# Load existing cache so we can *append* new sample points
# -----------------------------------------------------------
if os.path.isfile(points_data_cache):
    print("Loading cached point data …")
    with open(points_data_cache, "rb") as f:
        point_records = pickle.load(f)
    existing_coords = {(rec["latitude"], rec["longitude"]) for rec in point_records}
    next_id = max(rec["point_id"] for rec in point_records) + 1
else:
    point_records = []
    existing_coords = set()
    next_id = 0

print(f"Cache currently has {len(point_records)} points.")
added_this_run = 0

Loading cached point data …
Cache currently has 18897 points.


## Compute the Embeddings

(Note: would like to use Places365 but not available in Hugging Face yet, so using ViT base model instead)

In [6]:
device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# ---- Create final text embeddings ----
final_text_features = []
category_names = []  # so your categories keep their correct order

multi_prompts = {
    "C – Accommodation": [
        "a photo of a house or home",
        "an apartment building on a street",
        "houses in a residential neighborhood",
        "front view of a suburban house",
        "a cozy home exterior with a garden"
    ],
    "B – Industrial / Storage": [
        "a warehouse or big industrial building",
        "a factory with chimneys or machinery",
        "storage containers outside a building",
        "a logistics yard with trucks and crates",
        "industrial buildings in an urban area"
    ],
    "E – Commercial / Business / Service": [
        "a shop or cafe on the street",
        "a busy high street with stores",
        "a restaurant or small business front",
        "office building in the city",
        "people outside a retail store or service"
    ],
    "F – Local Community / Learning": [
        "a school or university building",
        "library or community centre",
        "children playing at a sports field",
        "outdoor playground or swimming pool",
        "museum, gallery or exhibition space"
    ]
}


# multi_prompts = {
#     "indoor": [
#         "a photo taken indoors",
#         "an indoor interior scene",
#         "inside a building",
#         "an indoor room photo"
#     ],
#     "terraced house": [
#         "a photo of a terraced house",
#         "a row of terraced homes",
#         "a UK terrace housing street",
#         "a brick terraced house"
#     ],
#     "semi-detached house": [
#         "a photo of a detached or semi detached house",
#         "a suburban detached house",
#         "a single-family home",
#         "a detached house on a residential street"
#     ],
#     "road": [
#         "a photo of a road",
#         "a street with cars or buildings",
#         "a roadway scene",
#         "a street-level view outdoors"
#     ],
#     "shop": [
#         "a photo of a shop",
#         "a store front",
#         "a retail business on a street",
#         "a commercial storefront"
#     ],
#     "car": [
#         "a photo dominated by the outside of a car",
#         "a vehicle exterior close-up",
#         "a photo of a parked car",
#         "a car on the street"
#     ],
#     "industrial": [
#         "a photo of an industrial building",
#         "a warehouse or factory building",
#         "an industrial site",
#         "a manufacturing facility"
#     ],
#     "wasteland": [
#         "a photo of wasteland or empty space",
#         "an abandoned empty outdoor area",
#         "a derelict vacant lot",
#         "an unused or empty land area"
#     ],
#     "greenery": [
#         "a photo of nice green space",
#         "a park or garden with trees",
#         "green plants and nature",
#         "a photo of natural greenery"]}


with torch.no_grad():
    for cat, prompts in multi_prompts.items():
        category_names.append(cat)

        # Encode all prompts for this category
        tokenized = clip.tokenize(prompts).to(device)
        txt_feats = model.encode_text(tokenized)

        # Normalise each embedding
        txt_feats = txt_feats / txt_feats.norm(dim=-1, keepdim=True)

        # Average across prompts
        avg_feat = txt_feats.mean(dim=0)

        # Normalise again (important!)
        avg_feat = avg_feat / avg_feat.norm()

        final_text_features.append(avg_feat.cpu())

# Convert to tensor of shape (num_categories, 512)
final_text_features = torch.stack(final_text_features)
print("Built improved category text embeddings:", final_text_features.shape)


def embed_and_score_clip(img_path):
    img = Image.open(img_path).convert("RGB")
    image_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        # Image embedding
        image_feat = model.encode_image(image_tensor)
        image_feat = image_feat / image_feat.norm(dim=-1, keepdim=True)

        # Compute similarity to averaged text embeddings
        sims = (image_feat @ final_text_features.to(device).T).softmax(dim=-1)

    return (
        image_feat.cpu().numpy()[0],   # 512D image embedding
        sims.cpu().numpy()[0]          # category probabilities
    )

# ------------------------------
# 4. Placeholder stats
# ------------------------------
def empty_attention_stats():
    return {"mean": None, "std": None, "entropy": None, "max_y": None, "max_x": None}

# ------------------------------
# 5. Embed all images
# ------------------------------
for rec in tqdm(point_records[:1000], desc="Embedding points", unit="point"):

    rec["embedding"] = []
    rec["category_scores"] = []
    rec["attn_stats"] = []
    rec["heatmap_paths"] = []

    for img_path in rec["image_files"]:

        # Use ORIGINAL images
        img_path = img_path.replace("airbnb-manchester/", "embeddings/").replace("../", "../../../")

        try:
            embedding, scores = embed_and_score_clip(img_path)

            rec["embedding"].append(embedding)
            rec["category_scores"].append(scores)
            rec["attn_stats"].append(empty_attention_stats())
            rec["heatmap_paths"].append(None)

        except Exception as e:
            tqdm.write(f"⚠️ Error: {e}")

output_file = (data_dir + "sample_points_cache/points_data_cache_with_CLIP_embeddings_and_scores_planninguseclasses_short.pkl")

with open(output_file, "wb") as f:
    pickle.dump(point_records, f)

print(f"\n💾 Saved embeddings + category scores for {len(point_records)} points.")

Built improved category text embeddings: torch.Size([4, 512])


Embedding points: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:20<00:00,  3.84point/s]



💾 Saved embeddings + category scores for 18897 points.
